In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
# Define image transformations for training and testing
train_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.975, 0.971, 0.974], [0.140, 0.148, 0.142]),
    ]
)

test_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.975, 0.971, 0.974], [0.140, 0.148, 0.142]),
    ]
)

In [3]:
# Directories for datasets
train_dir = "data_smiles/Training_Group"
augmented_train_dir = "data_smiles/Training_Group_Augmented"
test_dir = "data_smiles/Testing_Group"

# Combine the two training folders
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
augmented_train_dataset = datasets.ImageFolder(
    root=augmented_train_dir, transform=train_transforms
)

# Combine both datasets using torch.utils.data.ConcatDataset
combined_train_dataset = torch.utils.data.ConcatDataset(
    [train_dataset, augmented_train_dataset]
)

# Load the datasets into DataLoader
train_loader = DataLoader(combined_train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(
    datasets.ImageFolder(root=test_dir, transform=test_transforms),
    batch_size=32,
    shuffle=False,
)

# Training ResNet18

In [5]:
# Load a pretrained ResNet18 model
model = models.resnet18(weights=True)

# Modify the fully connected layer to output 2 classes (binary classification)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")

        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Statistics
            _, predicted = torch.max(outputs, 1)
            running_loss += loss.item()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total

        print(
            f"Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}"
        )

In [13]:
# Function to evaluate the model
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {correct / total:.4f}")

In [ ]:
# Train the model
train_model(model, criterion, optimizer, num_epochs=1)

In [ ]:
# Evaluate the model on the testing data
evaluate_model(model)